In [ ]:
pip install --quiet --upgrade tensorflow_federated

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

# TODO(b/148678573,b/148685415): must use the ReferenceExecutor because it
# supports unbounded references and tff.sequence_* intrinsics.
tff.framework.set_default_executor(tff.framework.ReferenceExecutor())

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

#all_buildings = pd.read_csv('/content/drive/My Drive/Dataset/building meta data.csv')
zipfile = np.load('/content/drive/My Drive/Dataset/Office_Cold_6000-9000_6-10_13.npz', allow_pickle=True)

Building_data = zipfile['data']
Building_data_norm = zipfile['data_norm']
Building_names = zipfile['name']
List_of_names = pd.Series(Building_names).unique()

NUM_EXAMPLES_PER_USER = 10000
BATCH_SIZE = 500

EC_min = np.min(Building_data[:,-1])
EC_max = np.max(Building_data[:,-1])
EC_mean = np.mean(Building_data[:,-1])

def get_data_for_name(Data, Names, name):
  output_sequence = []
  all_samples = [i for i, n in enumerate(Names) if n == name]

  for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
    batch_samples = all_samples[i:i+BATCH_SIZE]
    output_sequence.append({'x':np.array([Data[i,:-1] for i in batch_samples], dtype=np.float32),
                'y':np.array([Data[i,-1] for i in batch_samples], dtype=np.float32)})
  return output_sequence

def get_label_for_name(Data, Names, name):
  output_sequence = []
  all_samples = [i for i, n in enumerate(Names) if n == name]

  for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
    batch_samples = all_samples[i:i+BATCH_SIZE]
    output_sequence.append({'y':np.array([Data[i,-1] for i in batch_samples], dtype=np.float32)})
  return output_sequence


federated_train_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[:-2]]

federated_valid_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[-2:-1]]

#local_data = [get_data_for_name(Building_data_norm, Building_names, name) for name in List_of_names[-1:]]

#local_train_data = local_data[0][:2]

#local_test_data = local_data[0][-2:]

#Label_test_data = [get_label_for_name(Building_data, Building_names, name) for name in List_of_names[-1:]][0][-2:]

In [ ]:
BATCH_SPEC = collections.OrderedDict(x=tf.TensorSpec(shape=[None,6], dtype=tf.float32), y=tf.TensorSpec(shape=[None], dtype=tf.float32))

In [ ]:
num_layers = 1
num_neurons = 5
activation = 'relu'

def create_keras_model(num_layers, num_neurons, activation):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(num_neurons, activation=activation, input_shape=(6,)))
  for i in range(num_layers-1):
    model.add(tf.keras.layers.Dense(num_neurons, activation=activation))
  model.add(tf.keras.layers.Dense(1))
  return model

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model(num_layers=num_layers, num_neurons=num_neurons, activation=activation)
  return tff.learning.from_keras_model(keras_model, input_spec=BATCH_SPEC, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(6)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Dense(1)])
    
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(keras_model, input_spec=BATCH_SPEC, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
#initial_state = iterative_process.initialize()
best_metric = 80
for num_layers in [5]:
  for num_neurons in [5,10,15,20,25]:
    for activation in ['relu','sigmoid','tanh']:
      print('current parameters are: ', num_layers, num_neurons, activation)
      iterative_process = tff.learning.build_federated_averaging_process(
          model_fn,
          client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
          server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))
      state = iterative_process.initialize()
      NUM_ROUNDS = 100
      loss = []
      for round_num in range(NUM_ROUNDS):
        state, metrics = iterative_process.next(state, federated_train_data)
        evaluation = tff.learning.build_federated_evaluation(model_fn)
        valid_metrics = evaluation(state.model, federated_valid_data)
        loss.append(valid_metrics[1])
        if len(loss) > 1:
          if ((loss[-2]-loss[-1]) / loss[-2] < 0.005):
            print('Train process is done')
            break
      if valid_metrics[0] < best_metric:
        best_state = state
        best_metric = valid_metrics[0]
        best_num_layers = num_layers
        best_num_neurons = num_neurons
        best_activation = activation
print('best parameters are: ', best_num_layers, best_num_neurons, best_activation, best_metric)

In [ ]:
num_layers = 1
num_neurons = 5
activation = 'tanh'
NUM_ROUNDS = 50
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
str(iterative_process.initialize.type_signature)

In [ ]:
state = iterative_process.initialize()

In [ ]:
loss = []
for round_num in range(NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  evaluation = tff.learning.build_federated_evaluation(model_fn)
  valid_metrics = evaluation(state.model, federated_valid_data)
  loss.append(valid_metrics[1])
  if len(loss) > 1:
    if ((loss[-2]-loss[-1]) / loss[-2] < 0.005):
      print('Train process is done')
      break
  print('round {:2d}, metrics={}'.format(round_num, metrics), valid_metrics[0])

In [ ]:
loss

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
valid_metrics = evaluation(state.model, federated_valid_data)

In [ ]:
str(valid_metrics)

In [ ]:
forward_pass(batch_input=federated_train_data, training=False)
a = tff.learning.Model()

In [ ]:
local_model = create_keras_model(best_num_layers, best_num_neurons, best_activation)
local_fed_model = create_keras_model(best_num_layers, best_num_neurons, best_activation)

In [ ]:
tff.learning.assign_weights_to_keras_model(local_fed_model, best_state.model)
tf.keras.models.save_model(local_fed_model, '/content/drive/My Drive/Federated learning/Office_Cold_6000-9000_6-10_13')
tf.keras.models.save_model(local_model, '/content/drive/My Drive/Federated learning/Office_Cold_6000-9000_6-10_13_no_fed')
local_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
local_fed_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
local_fed_model.save('/content/drive/My Drive/Federated learning/Office_Cold_6000-9000_6-10_13_2.h5')
local_model.save('/content/drive/My Drive/Federated learning/Office_Cold_6000-9000_6-10_13_no_fed_2.h5')
#keras.models.load_model()

In [ ]:
local_model.evaluate(local_test_data[0]['x'], local_test_data[0]['y'], batch_size=32)

In [ ]:
local_fed_model.evaluate(local_test_data[0]['x'], local_test_data[0]['y'], batch_size=32)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto', baseline=None, restore_best_weights=True)
local_model.fit(local_train_data[0]['x'], local_train_data[0]['y'], batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping])
local_fed_model.fit(local_train_data[0]['x'], local_train_data[0]['y'], batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
p1 = local_fed_model.predict(local_test_data[1]['x'])
p2 = local_model.predict(local_test_data[1]['x'])
prediction1 = p1*(EC_max-EC_min)+EC_min
prediction2 = p1*(EC_max-EC_min)+EC_min
def CVRMSE(prediction, true):
  return (np.sqrt(np.mean(np.square(true - prediction))) / np.mean(true))

In [ ]:
CVRMSE(p1, local_test_data[0]['y'])